# Portfolio of Outstanding Loans Analysis

### Q1. Data as single dataframe

In [1]:
from pola import DataSet, MonthEndBalanceTabInfo, PaymentDueTabInfo, PaymentMadeTabInfo

In [2]:
loans_data = DataSet.from_excel(
    path = "2024 - Strat Casestudy.xlsx",
    static_tab="DATA-Static",
    data_tabs=[
        MonthEndBalanceTabInfo("DATA-Month End Balances"),
        PaymentDueTabInfo("DATA-Payment Due"),
        PaymentMadeTabInfo("DATA-Payment Made"),
    ],
) 
loans_data.data.head(6)

,loan_id,data,2016-01-31 00:00:00,2016-02-29 00:00:00,2016-03-31 00:00:00,2016-04-30 00:00:00,2016-05-31 00:00:00,2016-06-30 00:00:00,2016-07-31 00:00:00,2016-08-31 00:00:00,...,2022-10-31 00:00:00,2022-11-30 00:00:00,2022-12-31 00:00:00,origination_date,investor_1_acquisition_date,reversion_date,original_balance,product,pre_reversion_fixed_rate,post_reversion_boe_margin
0,1,Month End Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0000,0.000000,2020-12-31,2020-12-31,2022-12-31,150876,1,0.019881,0.0375
1,1,Payment Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2020-12-31,2020-12-31,2022-12-31,150876,1,0.019881,0.0375
2,1,Payment Due,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2020-12-31,2020-12-31,2022-12-31,150876,1,0.019881,0.0375
3,2,Month End Balance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,103580.0,103580.0000,103580.000000,2020-07-31,2020-12-31,2022-07-31,103580,1,0.019947,0.0375
4,2,Payment Made,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,517.9,582.6375,625.795833,2020-07-31,2020-12-31,2022-07-31,103580,1,0.019947,0.0375
5,2,Payment Due,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,517.9,582.6375,625.795833,2020-07-31,2020-12-31,2022-07-31,103580,1,0.019947,0.0375
